In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pickle

from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats
from statsmodels.stats.stattools import jarque_bera

from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

In [2]:
df = pd.read_csv('100_stocks_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,Open,High,Low,Close,Volume,Ticker,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,40.433960,41.368938,40.204048,41.192673,7564701,ADANIENT.NS,5.275391e+06,7564701,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,40.464615,41.192673,38.824574,39.138786,17188171,ADANIENT.NS,-7.351532e+06,-9623470,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.986048,-5.114644,-4.986048
2,38.778587,39.246078,37.475750,38.065861,11525782,ADANIENT.NS,-1.119344e+07,-21149252,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.741336,-2.779612,-7.590699
3,37.996887,39.069813,37.506405,38.632977,10660990,ADANIENT.NS,-6.490088e+06,-10488262,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.489828,1.478839,-6.213960
4,38.832237,39.069813,36.939289,37.084900,11002957,ADANIENT.NS,-1.598905e+07,-21491219,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.007138,-4.089635,-9.972096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241460,1529.200000,1543.000000,1515.600000,1523.350000,551554,UNITDSPR.NS,-9.265698e+07,435915083,0.150406,8.972093e+06,...,2.228798,2.065220,0.163578,-10.052454,-6.487623,-3.564831,1472.924483,-0.382553,-0.383287,-41.590460
241461,1533.900000,1552.900000,1525.400000,1547.250000,516123,UNITDSPR.NS,-9.235293e+07,436431206,0.207457,9.452557e+06,...,2.339432,2.120062,0.219370,-12.031591,-7.596416,-4.435175,1479.501082,1.568911,1.556730,-40.674067
241462,1547.000000,1559.250000,1540.800000,1555.750000,1224149,UNITDSPR.NS,-9.159323e+07,437655355,0.234416,9.588658e+06,...,2.442367,2.184523,0.257843,-6.196572,-7.316447,1.119875,1488.679001,0.549362,0.547858,-40.348153
241463,1560.500000,1561.950000,1525.050000,1537.750000,320652,UNITDSPR.NS,-9.169316e+07,437334703,0.206728,7.394316e+06,...,2.398303,2.227279,0.171024,-10.747593,-8.002677,-2.744917,1490.797494,-1.156998,-1.163744,-41.038324


In [3]:
stock_tickers = df['Ticker'].unique().tolist()

In [4]:
with open('all_data__list_of_dataframes.pkl', 'rb') as f:
    all_data = pickle.load(f)

In [5]:
def fill_nan_values(df):
    # Group by 'Ticker' to handle each stock separately
    return df.groupby('Ticker').apply(lambda x: x.fillna(method='bfill').fillna(method='ffill'))

In [6]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Ticker', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       'tr

In [7]:
def add_bb_rsi(df, bb_window=20, bb_std=2, rsi_window=14):
    # Bollinger Bands
    bb_indicator = BollingerBands(close=df['Close'], window=bb_window, window_dev=bb_std)
    df['bb_high'] = bb_indicator.bollinger_hband()
    df['bb_low'] = bb_indicator.bollinger_lband()
    df['bb_mid'] = bb_indicator.bollinger_mavg()
    
    # RSI
    rsi_indicator = RSIIndicator(close=df['Close'], window=rsi_window)
    df['rsi'] = rsi_indicator.rsi()
    
    return df

df = add_bb_rsi(df)
df = fill_nan_values(df)

In [8]:
def calculate_true_range(df):
    df['TR'] = np.maximum(df['High'] - df['Low'], 
                          np.maximum(abs(df['High'] - df['Close'].shift(1)),
                                     abs(df['Low'] - df['Close'].shift(1))))
    return df

def coiled_spring_nr7(df):
    df['ATR'] = df['TR'].rolling(window=7).mean()
    df['NR7'] = df['TR'].rolling(window=7).min() == df['TR']
    
    df['long_signal'] = (df['NR7'] & (df['Close'] > df['bb_mid']) & (df['rsi'] < 50))
    df['short_signal'] = (df['NR7'] & (df['Close'] < df['bb_mid']) & (df['rsi'] > 50))
    
    return df

def finger_finder(df, atr_period=14, atr_multiplier=2):
    df['ATR'] = df['TR'].rolling(window=atr_period).mean()
    df['Upper_Band'] = df['High'].rolling(window=2).max() + (df['ATR'] * atr_multiplier)
    df['Lower_Band'] = df['Low'].rolling(window=2).min() - (df['ATR'] * atr_multiplier)
    
    df['long_signal'] = (df['Close'] > df['Upper_Band'].shift(1)) & (df['rsi'] < 70)
    df['short_signal'] = (df['Close'] < df['Lower_Band'].shift(1)) & (df['rsi'] > 30)
    
    return df

def power_spike(df, volume_threshold=2):
    df['volume_ma'] = df['Volume'].rolling(window=20).mean()
    df['price_change'] = df['Close'] - df['Open']
    
    df['long_signal'] = (df['Volume'] > df['volume_ma'] * volume_threshold) & (df['price_change'] > 0) & (df['rsi'] < 70)
    df['short_signal'] = (df['Volume'] > df['volume_ma'] * volume_threshold) & (df['price_change'] < 0) & (df['rsi'] > 30)
    
    return df

In [9]:
def backtest_strategy(df, initial_capital=100000, transaction_cost_pct=0.001):
    df['position'] = np.where(df['long_signal'], 1, np.where(df['short_signal'], -1, 0))
    df['position'] = df['position'].fillna(method='ffill')
    
    df['returns'] = df['Close'].pct_change()
    df['strategy_returns'] = df['position'].shift(1) * df['returns'] - abs(df['position'].diff()) * transaction_cost_pct
    
    df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()
    df['cumulative_strategy'] = initial_capital * df['cumulative_returns']
    
    total_return = df['cumulative_returns'].iloc[-1] - 1
    sharpe_ratio = np.sqrt(252) * df['strategy_returns'].mean() / df['strategy_returns'].std()
    max_drawdown = (df['cumulative_strategy'] / df['cumulative_strategy'].cummax() - 1).min()
    
    return {
        'Total Return': total_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Final Portfolio Value': df['cumulative_strategy'].iloc[-1]
    }

def process_stock(df, strategy_func):
    df = add_bb_rsi(df)
    df = calculate_true_range(df)
    df = fill_nan_values(df)
    df = strategy_func(df)
    results = backtest_strategy(df)
    return results

strategies = {
    'Coiled Spring NR7': coiled_spring_nr7,
    'Finger Finder': finger_finder,
    'Power Spike': power_spike
}

In [10]:
results = {}

for strategy_name, strategy_func in strategies.items():
    strategy_results = {}
    for df in all_data:
        ticker = df['Ticker'].iloc[0]  # Extract the ticker from the DataFrame
        stock_results = process_stock(df, strategy_func)
        strategy_results[ticker] = stock_results
    results[strategy_name] = strategy_results

# Print results
for strategy, stocks in results.items():
    print(f"\nResults for {strategy}:")
    for ticker, performance in stocks.items():
        print(f"{ticker}: {performance}")

# Results converted to dataframe
results_df = pd.DataFrame({(strategy, ticker): performance 
                           for strategy, stocks in results.items() 
                           for ticker, performance in stocks.items()})
print("\nResults DataFrame:")
print(results_df)


Results for Coiled Spring NR7:
ADANIENT.NS: {'Total Return': -0.10371573181926941, 'Sharpe Ratio': -0.18650802152202975, 'Max Drawdown': -0.17742283911519252, 'Final Portfolio Value': 89628.42681807306}
ADANIPORTS.NS: {'Total Return': -0.13861850205264448, 'Sharpe Ratio': -0.2909044792510182, 'Max Drawdown': -0.176235689888685, 'Final Portfolio Value': 86138.14979473555}
APOLLOHOSP.NS: {'Total Return': -0.08499160823852348, 'Sharpe Ratio': -0.22941084847125812, 'Max Drawdown': -0.10865264506338257, 'Final Portfolio Value': 91500.83917614765}
ASIANPAINT.NS: {'Total Return': -0.15399972059232414, 'Sharpe Ratio': -0.43489992158269913, 'Max Drawdown': -0.17514540954775704, 'Final Portfolio Value': 84600.02794076758}
AXISBANK.NS: {'Total Return': -0.2706899671329335, 'Sharpe Ratio': -0.7641583228920122, 'Max Drawdown': -0.2975883467525774, 'Final Portfolio Value': 72931.00328670665}
BAJAJ-AUTO.NS: {'Total Return': 0.011677862283091534, 'Sharpe Ratio': 0.059397448923369935, 'Max Drawdown': 

In [11]:
results_df.columns

MultiIndex([('Coiled Spring NR7',   'ADANIENT.NS'),
            ('Coiled Spring NR7', 'ADANIPORTS.NS'),
            ('Coiled Spring NR7', 'APOLLOHOSP.NS'),
            ('Coiled Spring NR7', 'ASIANPAINT.NS'),
            ('Coiled Spring NR7',   'AXISBANK.NS'),
            ('Coiled Spring NR7', 'BAJAJ-AUTO.NS'),
            ('Coiled Spring NR7', 'BAJFINANCE.NS'),
            ('Coiled Spring NR7', 'BAJAJFINSV.NS'),
            ('Coiled Spring NR7',       'BPCL.NS'),
            ('Coiled Spring NR7', 'BHARTIARTL.NS'),
            ...
            (      'Power Spike',        'SRF.NS'),
            (      'Power Spike',  'TATAPOWER.NS'),
            (      'Power Spike', 'TORNTPHARM.NS'),
            (      'Power Spike',      'TRENT.NS'),
            (      'Power Spike',   'TVSMOTOR.NS'),
            (      'Power Spike',        'VBL.NS'),
            (      'Power Spike',       'VEDL.NS'),
            (      'Power Spike',     'ZOMATO.NS'),
            (      'Power Spike',  'ZYDUSLIFE.NS

In [12]:
coiled_spring_nr7_results_df = pd.DataFrame(results_df['Coiled Spring NR7'])
coiled_spring_nr7_results_df.to_csv('coiled_spring_nr7_results_df.csv')

finger_finder_results_df = pd.DataFrame(results_df['Finger Finder'])
finger_finder_results_df.to_csv('finger_finder_results_df.csv')

power_spike_results_df = pd.DataFrame(results_df['Power Spike'])
power_spike_results_df.to_csv('power_spike_results_df.csv')

In [13]:
coiled_spring_nr7_results_df

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJFINANCE.NS,BAJAJFINSV.NS,BPCL.NS,BHARTIARTL.NS,...,SRF.NS,TATAPOWER.NS,TORNTPHARM.NS,TRENT.NS,TVSMOTOR.NS,VBL.NS,VEDL.NS,ZOMATO.NS,ZYDUSLIFE.NS,UNITDSPR.NS
Total Return,-0.103716,-0.138619,-0.084992,-0.154000,-0.270690,0.011678,-0.272608,-0.047720,-0.051405,-0.196732,...,-0.127117,-0.083827,-0.160278,-0.139862,-0.159567,-0.051838,-0.289966,0.111574,-0.138408,-0.094390
Sharpe Ratio,-0.186508,-0.290904,-0.229411,-0.434900,-0.764158,0.059397,-0.681549,-0.086847,-0.120636,-0.646510,...,-0.334972,-0.227889,-0.491054,-0.313672,-0.256735,-0.084526,-0.603191,0.393804,-0.514436,-0.231212
Max Drawdown,-0.177423,-0.176236,-0.108653,-0.175145,-0.297588,-0.053212,-0.287823,-0.161338,-0.087196,-0.201356,...,-0.181869,-0.124286,-0.192114,-0.165212,-0.170475,-0.213059,-0.295260,-0.068821,-0.163458,-0.154371
Final Portfolio Value,89628.426818,86138.149795,91500.839176,84600.027941,72931.003287,101167.786228,72739.184477,95228.030363,94859.463566,80326.782359,...,87288.298325,91617.281128,83972.228005,86013.817057,84043.261288,94816.196410,71003.443237,111157.428461,86159.153656,90561.015210


In [14]:
finger_finder_results_df

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJFINANCE.NS,BAJAJFINSV.NS,BPCL.NS,BHARTIARTL.NS,...,SRF.NS,TATAPOWER.NS,TORNTPHARM.NS,TRENT.NS,TVSMOTOR.NS,VBL.NS,VEDL.NS,ZOMATO.NS,ZYDUSLIFE.NS,UNITDSPR.NS
Total Return,0.065314,-0.157168,0.050432,0.073819,0.030143,0.051173,-0.033129,-0.011161,0.039228,0.011771,...,0.008338,-0.173397,-0.073493,0.008872,0.000636,-0.037843,0.077278,0.003988,0.096767,0.040718
Sharpe Ratio,0.240577,-0.430737,0.235183,0.293931,0.167282,0.290830,-0.068259,-0.024686,0.212923,0.051559,...,0.042307,-0.655430,-0.265280,0.084467,0.011932,-0.227619,0.260958,0.051485,0.497004,0.350144
Max Drawdown,-0.033912,-0.189119,-0.037431,-0.002738,-0.043532,-0.008851,-0.119961,-0.046567,-0.024021,-0.069603,...,-0.058258,-0.182734,-0.109540,-0.031049,-0.047529,-0.054841,-0.032565,-0.050485,-0.001000,-0.016275
Final Portfolio Value,106531.449661,84283.231547,105043.168866,107381.880459,103014.251277,105117.261931,96687.079285,98883.877060,103922.768055,101177.086610,...,100833.838514,82660.318361,92650.703389,100887.218185,100063.573224,96215.659994,107727.774091,100398.823974,109676.693217,104071.772364


In [15]:
power_spike_results_df

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJFINANCE.NS,BAJAJFINSV.NS,BPCL.NS,BHARTIARTL.NS,...,SRF.NS,TATAPOWER.NS,TORNTPHARM.NS,TRENT.NS,TVSMOTOR.NS,VBL.NS,VEDL.NS,ZOMATO.NS,ZYDUSLIFE.NS,UNITDSPR.NS
Total Return,0.431056,-0.546845,-0.179057,0.140963,0.082045,-0.067972,-0.302673,-0.107040,0.045947,0.521207,...,-0.274127,-0.266429,-0.364326,-0.375707,0.247623,-0.313362,-0.022431,-0.195368,-0.035218,0.064399
Sharpe Ratio,0.268074,-0.564394,-0.129671,0.240552,0.135795,-0.081279,-0.391204,-0.104539,0.091988,0.539918,...,-0.217907,-0.288516,-0.392671,-0.417894,0.262354,-0.378928,0.037413,-0.319292,0.009755,0.107727
Max Drawdown,-0.457457,-0.550154,-0.482913,-0.125511,-0.162707,-0.262594,-0.330757,-0.216097,-0.183755,-0.140436,...,-0.446193,-0.376442,-0.507723,-0.400997,-0.338182,-0.325395,-0.352024,-0.407883,-0.259096,-0.233309
Final Portfolio Value,143105.576434,45315.452318,82094.304020,114096.333539,108204.492614,93202.801388,69732.716413,89296.034966,104594.721313,152120.672650,...,72587.312893,73357.084766,63567.415435,62429.314510,124762.261238,68663.799107,97756.943302,80463.216076,96478.179808,106439.894633
